In [9]:
import random

class MemoryTier:
    def __init__(self, name, size, latency):
        self.name = name  # Name of the memory tier (e.g., DRAM, NVM)
        self.size = size  # Total capacity of the memory tier
        self.latency = latency  # Access latency of the memory tier (in ms)
        self.data = {}  # Storage for the memory content (key-value pairs)
        self.access_order = []  # Track access order for eviction (LRU policy)

    def access(self, address):
        """Access data and update access order."""
        if address in self.data:
            # Update access order to mark as recently used
            if address in self.access_order:
                self.access_order.remove(address)
            self.access_order.append(address)
            return self.data[address], self.latency
        return None, self.latency  # Return None if address not found

    def write(self, address, value):
        """Write data and evict if memory is full."""
        if len(self.data) >= self.size:
            evicted_address = self.access_order.pop(0)  # Evict least recently used (LRU)
            del self.data[evicted_address]
            print(f"Evicted address {evicted_address} from {self.name}")
        self.data[address] = value
        self.access_order.append(address)

class TieredMemorySystem:
    def __init__(self, dram_size, dram_latency, dram_bandwidth, nvm_size, nvm_latency, nvm_bandwidth):
        self.dram = MemoryTier("DRAM", dram_size, dram_latency)
        self.nvm = MemoryTier("NVM", nvm_size, nvm_latency)
        self.dram_bandwidth = dram_bandwidth  # In MB/s
        self.nvm_bandwidth = nvm_bandwidth  # In MB/s
        self.access_count = {}
        self.threshold = 2  # Reduced threshold for triggering migration
        self.total_copy_time = 0
        self.total_remap_time = 0
        self.migration_count = 0

    def access_memory(self, address, value=None, write=False):
        # Access DRAM first
        data, latency = self.dram.access(address)
        if data is not None:
            print(f"Address {address} found in DRAM. Latency: {latency} ms")
            return latency

        # Access NVM if not found in DRAM
        data, latency = self.nvm.access(address)
        if data is not None:
            print(f"Address {address} found in NVM. Latency: {latency} ms")
            self.access_count[address] = self.access_count.get(address, 0) + 1
            print(f"Access count for address {address}: {self.access_count[address]}")
            if self.access_count[address] > self.threshold:
                self.migrate_to_dram(address, data)
            return latency

        # If not found, write to NVM
        if write:
            print(f"Address {address} not found. Writing to NVM.")
            self.nvm.write(address, value)
        return latency

    def migrate_to_dram(self, address, value):
        try:
            # Simulate memory copying time
            data_size = 1024  # Assume 1 KB per page
            copy_time = data_size / (self.dram_bandwidth * 1024 / 1000)  # Time in ms
            remap_time = 0.05  # Fixed remapping time in ms
            self.total_copy_time += copy_time
            self.total_remap_time += remap_time
            self.migration_count += 1

            # Perform migration
            self.dram.write(address, value)
            del self.nvm.data[address]
            print(f"Migrated address {address} to DRAM (Copy Time: {copy_time:.2f} ms, Remap Time: {remap_time:.2f} ms).")
        except MemoryError:
            print("DRAM is full. Cannot migrate.")

    def get_performance_metrics(self):
        return {
            "Total Copy Time (ms)": self.total_copy_time,
            "Total Remap Time (ms)": self.total_remap_time,
            "Total Migrations": self.migration_count,
        }




In [10]:
if __name__ == "__main__":
    # Initialize the tiered memory system
    tiered_memory = TieredMemorySystem(
        dram_size=10, 
        dram_latency=0.1, 
        dram_bandwidth=2400,  # DRAM bandwidth: 2400 MB/s
        nvm_size=100, 
        nvm_latency=0.5, 
        nvm_bandwidth=128  # NVM bandwidth: 128 MB/s
    )

    # Simulate skewed memory accesses
    hot_addresses = [random.randint(0, 19) for _ in range(20)]
    for _ in range(500):  # Increased simulation steps
        if random.random() < 0.7:
            addr = random.choice(hot_addresses)  # Skewed access to hot addresses
        else:
            addr = random.randint(0, 99)
        tiered_memory.access_memory(addr, value=random.randint(0, 100), write=True)
        tiered_memory.access_memory(addr)

    # Get and print performance metrics
    metrics = tiered_memory.get_performance_metrics()
    print("Performance Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")

Address 38 not found. Writing to NVM.
Address 38 found in NVM. Latency: 0.5 ms
Access count for address 38: 1
Address 15 not found. Writing to NVM.
Address 15 found in NVM. Latency: 0.5 ms
Access count for address 15: 1
Address 2 not found. Writing to NVM.
Address 2 found in NVM. Latency: 0.5 ms
Access count for address 2: 1
Address 3 not found. Writing to NVM.
Address 3 found in NVM. Latency: 0.5 ms
Access count for address 3: 1
Address 0 not found. Writing to NVM.
Address 0 found in NVM. Latency: 0.5 ms
Access count for address 0: 1
Address 79 not found. Writing to NVM.
Address 79 found in NVM. Latency: 0.5 ms
Access count for address 79: 1
Address 16 not found. Writing to NVM.
Address 16 found in NVM. Latency: 0.5 ms
Access count for address 16: 1
Address 1 not found. Writing to NVM.
Address 1 found in NVM. Latency: 0.5 ms
Access count for address 1: 1
Address 30 not found. Writing to NVM.
Address 30 found in NVM. Latency: 0.5 ms
Access count for address 30: 1
Address 19 not found. 